In [4]:
import nbimporter
from template_manager import template_selector
from patient_data_extraction import extract_patient_info,extract_patient_measurements,image_extractor,mot_extractor
from file_conversion import cargar_path
from aux_calculations import calc_e_e,remove_signs,expand_dict_with_lists_inplace
from docx import Document
from docxtpl import DocxTemplate

def render_template(path)-> None:
    '''
    acepta un path y determina si es cardiaco o stress, si son estos extrae medidas, tratornos de motilidad y mapa polar
    Si no lo es extrae solamente datos del paciente y las imagenes del archivo docx producido por el equipo vinno
    guarda la renderizaci'on en la misma carpeta donde estaba el docx para poder realizar una comparacion
    
    '''
    doc=Document(path)
    template,tipo=template_selector(path)
    info_pac= extract_patient_info(doc)
    image=image_extractor(doc,template,tipo=tipo)
    #guardo el save path con nombre tipo de estudio y fecha en la misma carpeta.
    save_path=str(path).rsplit('\\',1)[0]+f'\\{info_pac["Name"]} {tipo} {info_pac["Exam Date"]}.docx'
    print(save_path)
    print(tipo)
    #Si es carotida solo van los datos del paciente
    if tipo in ['card','stress']: 
        medidas= extract_patient_measurements(path) 
        if tipo =='stress':
            if isinstance(medidas.get('MV Vel E',None),list):
                expand_dict_with_lists_inplace(medidas)
                medidas['E_e_rel'],medidas['e_e_avg']=calc_e_e(medidas)
            mot=mot_extractor(doc)
            context={**info_pac,**medidas,'image': image['image'], 'mot': mot['mot']}
        
        else:
            context={**info_pac,**medidas,'image': image['image']}
    else: 
        context={**info_pac,'image': image['image']}
        
    #quito los espacios en los context para que no haya conflicto
    context= remove_signs(context)
    template.render(context)
    template.save(save_path)
    print(f'informe {info_pac["Name"]} creado con exito')
        

In [2]:
lista_de_trabajo=cargar_path(r"C:\Users\Luciano\Documents\temp\documentos")
lista_de_trabajo

Converted C:\Users\Luciano\Documents\temp\documentos\[CEFERINAIBANEZ]-[20241007004]-[7a431e1f-04dc-4a09-9056-2660347709d4]\5_F4-12L_Carotid.Report.V3.doc to C:\Users\Luciano\Documents\temp\documentos\[CEFERINAIBANEZ]-[20241007004]-[7a431e1f-04dc-4a09-9056-2660347709d4]\5_F4-12L_Carotid.Report.V3.docx
Converted C:\Users\Luciano\Documents\temp\documentos\[CEFERINAIBANEZ]-[20241007004]-[7a431e1f-04dc-4a09-9056-2660347709d4]\6_S1-6P_Lucho Card.Report.V3.doc to C:\Users\Luciano\Documents\temp\documentos\[CEFERINAIBANEZ]-[20241007004]-[7a431e1f-04dc-4a09-9056-2660347709d4]\6_S1-6P_Lucho Card.Report.V3.docx
Converted C:\Users\Luciano\Documents\temp\documentos\[CEFERINAIBANEZ]-[20241007004]-[7a431e1f-04dc-4a09-9056-2660347709d4]\7_S1-6P_Lucho Card.Report.V3.doc to C:\Users\Luciano\Documents\temp\documentos\[CEFERINAIBANEZ]-[20241007004]-[7a431e1f-04dc-4a09-9056-2660347709d4]\7_S1-6P_Lucho Card.Report.V3.docx
Converted C:\Users\Luciano\Documents\temp\documentos\[corinalopez]-[20241008013]-[59ae

[WindowsPath('C:/Users/Luciano/Documents/temp/documentos/[CEFERINAIBANEZ]-[20241007004]-[7a431e1f-04dc-4a09-9056-2660347709d4]/5_F4-12L_Carotid.Report.V3.docx'),
 WindowsPath('C:/Users/Luciano/Documents/temp/documentos/[CEFERINAIBANEZ]-[20241007004]-[7a431e1f-04dc-4a09-9056-2660347709d4]/6_S1-6P_Lucho Card.Report.V3.docx'),
 WindowsPath('C:/Users/Luciano/Documents/temp/documentos/[CEFERINAIBANEZ]-[20241007004]-[7a431e1f-04dc-4a09-9056-2660347709d4]/7_S1-6P_Lucho Card.Report.V3.docx'),
 WindowsPath('C:/Users/Luciano/Documents/temp/documentos/[corinalopez]-[20241008013]-[59ae336b-c9b6-40e9-9328-725cc1ba9404]/21_F4-12L_Carotid.Report.V3.docx'),
 WindowsPath('C:/Users/Luciano/Documents/temp/documentos/[del carmebnevada]-[20241008007]-[8bdb682f-15c2-43a9-b746-c237209e72eb]/13_S1-6P_Lucho Card.Report.V3.docx'),
 WindowsPath('C:/Users/Luciano/Documents/temp/documentos/[del carmengambarte]-[20241008008]-[4ece95a0-abb6-47d9-bca4-4df518ddcc16]/14_S1-6P_Lucho Card.Report.V3.docx'),
 WindowsPath('

In [6]:
for i in lista_de_trabajo:
    render_template(i)

C:\Users\Luciano\Documents\temp\documentos\[CEFERINAIBANEZ]-[20241007004]-[7a431e1f-04dc-4a09-9056-2660347709d4]\IBANEZ  CEFERINA carotid 08-10-2024.docx
carotid
informe IBANEZ  CEFERINA creado con exito
C:\Users\Luciano\Documents\temp\documentos\[CEFERINAIBANEZ]-[20241007004]-[7a431e1f-04dc-4a09-9056-2660347709d4]\IBANEZ  CEFERINA stress 08-10-2024.docx
stress
informe IBANEZ  CEFERINA creado con exito
C:\Users\Luciano\Documents\temp\documentos\[CEFERINAIBANEZ]-[20241007004]-[7a431e1f-04dc-4a09-9056-2660347709d4]\IBANEZ  CEFERINA stress 08-10-2024.docx
stress
informe IBANEZ  CEFERINA creado con exito
C:\Users\Luciano\Documents\temp\documentos\[corinalopez]-[20241008013]-[59ae336b-c9b6-40e9-9328-725cc1ba9404]\lopez marta corina carotid 08-10-2024.docx
carotid
informe lopez marta corina creado con exito
C:\Users\Luciano\Documents\temp\documentos\[del carmebnevada]-[20241008007]-[8bdb682f-15c2-43a9-b746-c237209e72eb]\nevada maria del carmeb stress 08-10-2024.docx
stress
informe nevada mar